In [1]:
# from gensim.models import Word2Vec
from sklearn import preprocessing
import nltk

def REL_embedding(corpus_dict):
    le = preprocessing.LabelEncoder()
    RELSet = set()
    for k,v in corpus_dict.items():
        if v!="Other":
            rel=v.split("(")
            relation=rel[0]
            RELSet.add(relation)
    RELSet.add("O")
    li = list(RELSet) 
    trans = le.fit_transform(li)
    
    print(trans)
    reldict = {}
    for i in range(len(li)):
        reldict[li[i]] = trans[i]
    return reldict
# REL_embedding(corpus_dict)

In [ ]:
from sklearn import preprocessing
import nltk

def RELWithDir_embedding(corpus_dict):
    le = preprocessing.LabelEncoder()
    RELSet = set()
    for k,v in corpus_dict.items():
        if v!="Other":
            RELSet.add(v)
    RELSet.add("O")
    li = list(RELSet) 
    trans = le.fit_transform(li)
    
    reldict = {}
    for i in range(len(li)):
        reldict[li[i]] = trans[i]
    return reldict

In [2]:
from sklearn import preprocessing
import nltk

def POS_embedding():
    le = preprocessing.LabelEncoder()
    set1 = {"CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNS","NNP","NNPS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB"}
    li = list(set1) 
    li1 = []
    li1.append(li)
    trans = le.fit_transform(li)

    posEmbDict = {}
    for i in range(len(li)):
        posEmbDict[li[i]] = trans[i]
    return posEmbDict
# POS_embedding()

In [3]:
from sklearn import preprocessing
import nltk

def depParsingEmbedding():
    le = preprocessing.LabelEncoder()
    set1 = {'ROOT', 'acl', 'acomp', 'advcl', 'advmod', 'agent', 'amod', 'appos', 'attr', 'aux', 'auxpass', 'case', 'cc', 'ccomp', 'compound', 'conj', 'csubj', 'csubjpass', 'dative', 'dep', 'det', 'dobj', 'expl', 'intj', 'mark', 'meta', 'neg', 'nmod', 'npadvmod', 'nsubj', 'nsubjpass', 'nummod', 'oprd', 'parataxis', 'pcomp', 'pobj', 'poss', 'preconj', 'predet', 'prep', 'prt', 'punct', 'quantmod', 'relcl', 'xcomp','-'}
    li = list(set1) 
    li1 = []
    li1.append(li)
    trans = le.fit_transform(li)

    depParseEmbDict = {}
    for i in range(len(li)):
        depParseEmbDict[li[i]] = trans[i]
    return depParseEmbDict
# depParsingEmbedding()

In [4]:
import nltk
from sklearn import preprocessing

def NERTagEmbedding():
    le = preprocessing.LabelEncoder()
    set1 = {"PERSON","NORP","FAC","ORG","GPE","LOC","PRODUCT","EVENT","WORK_OF_ART","LAW","LANGUAGE","DATE","TIME","PERCENT","MONEY","QUANTITY","ORDINAL","CARDINAL","NaN","None"}
    li = list(set1) 
    li1 = []
    li1.append(li)
    trans = le.fit_transform(li)

    nerTagEmbDict = {}
    for i in range(len(li)):
        nerTagEmbDict[li[i]] = trans[i]
    return nerTagEmbDict
# NERTagEmbedding()

In [5]:
from nltk.corpus import wordnet as wn
def find_hypernyms(word):
    hypernyms=[]
    for ss in wn.synsets('green'):
        for hyper in ss.hypernyms():
            hypernyms.append(hyper.name())
    return (hypernyms)

In [5]:
import spacy
nlp = spacy.load("en_core_web_sm")
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet as wn
def Corpus_dictFormation(f):
    d=(f.read())
    data=d.split("\n\n\n")
    data=data[:len(data)-1]
    import re
    for i in range(len(data)):
        data[i]=re.sub(r"(\d+\t)"," ",data[i])
        data[i]=re.sub('"',' ',data[i])

    corpus_dict={}
    for i in range(len(data)):
    #     print(i)
        temp=data[i].split("\n")

        corpus_dict[temp[0]]=temp[1]
    return corpus_dict

def Corpus_Reading(corpus_dict):

    ind=0
    MainDict1={}
    relList = []
    dirList = []
    wholeRelList = []
    for k,v in corpus_dict.items():
        wholeRelList.append(v)
        if ind not in MainDict1.keys():
            MainDict1[ind]={'token':{},'POSTAG':{},'NER':{},'DEPPARSE':{},'Direction':"","REL":"","e1Start":0,"e1End":0,"e2Start":0,"e2End":0}
            if v!="Other":
                rel=v.split("(")
                rel1=rel[0]
                relList.append(rel1)
                dire=rel[1].split(",")
                if dire[0]=="e1":
                    MainDict1[ind]["Direction"]="+1"
                    MainDict1[ind]["REL"]=rel1
                    dirList.append(1)
                else:
                    MainDict1[ind]["Direction"]="-1"
                    MainDict1[ind]["REL"]=rel1
                    dirList.append(-1)
            else:
                MainDict1[ind]["Direction"]="0"
                MainDict1[ind]["REL"]="O"
                relList.append(v)
                dirList.append(0)


            tempSent=k
            tempSent=re.sub('<e1>|</e1>|<e2>|</e2>|"',"",tempSent).lstrip()
            tempSent=re.sub('-|/'," ",tempSent)

            tokens=nltk.word_tokenize(tempSent)
            for i in range(len(tokens)):
                MainDict1[ind]["token"][i]=tokens[i]

            nlpdoc = nlp(tempSent)
            for nd in nlpdoc:
        #         print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,token.shape_, token.is_alpha, token.is_stop)
                if nd.text.isspace()==False and nd.text in tokens:
                    MainDict1[ind]['DEPPARSE'][tokens.index(nd.text)]=nd.dep_


            e1=re.findall("<e1>.*?</e1>",k)
            e2=re.findall("<e2>.*?</e2>",k)
            e1=re.sub("<e1>|</e1>","",e1[0])
      
            e1=re.sub(r"([a-z])\-([a-z])", r"\1 \2", e1 , 0, re.IGNORECASE)
            e2=re.sub("<e2>|</e2>","",e2[0])
       
            e2=re.sub(r"([a-z])\-([a-z])", r"\1 \2", e2 , 0, re.IGNORECASE)
            e1_list=e1.split(" ")
            e2_list=e2.split(" ")
    
            MainDict1[ind]["e1Start"]=tokens.index(e1_list[0])
            MainDict1[ind]["e1End"]=tokens.index(e1_list[-1])
            MainDict1[ind]["e2Start"]=tokens.index(e2_list[0])
            MainDict1[ind]["e2End"]=tokens.index(e2_list[-1])
        
            posTags=nltk.pos_tag(tokens)
            for i in range(len(tokens)):
                MainDict1[ind]["POSTAG"][i]=posTags[i][1]


    
            spacy_sent1=" ".join(e1_list)
            spacy_sent2=" ".join(e2_list)
   
            doc1=nlp(spacy_sent1)
            doc2=nlp(spacy_sent2)
   
            l1,l2=0,0
            for i in tokens:
                    MainDict1[ind]['NER'][tokens.index(i)]="None"

            for e1 in doc1.ents:
                l1=e1.label_

            for e2 in doc2.ents:
                l2=e2.label_


            for i in tokens:
                if i in e1_list and MainDict1[ind]['NER'][tokens.index(i)]=="None":
                    if l1==0:
                        MainDict1[ind]['NER'][tokens.index(i)]="NaN"
                    else:
                        MainDict1[ind]['NER'][tokens.index(i)]=l1
                elif i in e2_list and MainDict1[ind]['NER'][tokens.index(i)]=="None":
                    if l2==0:
                        MainDict1[ind]['NER'][tokens.index(i)]="NaN"
                    else:
                        MainDict1[ind]['NER'][tokens.index(i)]=l2

            tempSent=""  
            ind+=1
    MainDict2={}
    ind=0

    for k,v in corpus_dict.items():
        if ind not in MainDict2.keys():
            MainDict2[ind]={'token':[],'POSTAG':[],'NER':[],'DEPPARSE':[],'Direction':"","REL":"","e1Start":0,"e1End":0,"e2Start":0,"e2End":0,"lemma":[]}

            for k1,v1 in MainDict1[ind]['token'].items():
                 MainDict2[ind]['token'].append(v1)

            lemmatizer = WordNetLemmatizer()
            for i in MainDict2[ind]['token']:
                MainDict2[ind]['lemma'].append(lemmatizer.lemmatize(i))
                
#             for i in MainDict2[ind]['token']:
#                 tempHyper=find_hypernyms(i) 
#                 MainDict2[ind]['hypernyms'].append(tempHyper)
                
            for k1,v1 in MainDict1[ind]['POSTAG'].items():
                MainDict2[ind]['POSTAG'].append(v1)
            for k1,v1 in MainDict1[ind]['NER'].items():
                MainDict2[ind]['NER'].append(v1)
            for k1,v1 in MainDict1[ind]['DEPPARSE'].items():
                MainDict2[ind]['DEPPARSE'].append(v1)
            MainDict2[ind]['REL']=MainDict1[ind]['REL']

            MainDict2[ind]['Direction']=int(MainDict1[ind]['Direction'])
            MainDict2[ind]['e1Start']=int(MainDict1[ind]['e1Start'])
            MainDict2[ind]['e1End']=int(MainDict1[ind]['e1End'])
            MainDict2[ind]['e2Start']=int(MainDict1[ind]['e2Start'])
            MainDict2[ind]['e2End']=int(MainDict1[ind]['e2End'])
            ind+=1
            
    MainDictEmbedding={}
    ind=0
    nerTagEmbDict=NERTagEmbedding()
    depParseEmbDict=depParsingEmbedding()
    posEmbDict=POS_embedding()
    reldict=REL_embedding(corpus_dict)

    for k,v in corpus_dict.items():
        if ind not in MainDictEmbedding.keys():
            MainDictEmbedding[ind]={'token':[],'POSTAG':[],'NER':[],'DEPPARSE':[],'Direction':"","REL":"","e1Start":0,"e1End":0,"e2Start":0,"e2End":0,"lemma":[]}

            tempSent=k
            tempSent=re.sub('<e1>|</e1>|<e2>|</e2>|"',"",tempSent).lstrip()
            tempSent=re.sub('-|/'," ",tempSent)

            tokens=nltk.word_tokenize(tempSent)
            for i in range(len(tokens)):
                MainDict1[ind]["token"][i]=tokens[i]
            tokenli=[]
            tokenli.append(tokens)
            model = Word2Vec(tokenli, min_count=1,size=1)
            words = list(model.wv.vocab)
            for i in words:
                if i in tokens:
                    (MainDictEmbedding[ind]['token']).append(int(model[i][0]*1000))
            MainDictEmbedding[ind]['token']=np.std(MainDictEmbedding[ind]['token'])
            
#             hyperEmb=MainDict2[ind]['hypernyms']
#             model2 = Word2Vec(hyperEmb, min_count=1,size=1)
#             words2 = list(model2.wv.vocab)
#             for i in words2:
#                 if i in hyperEmb:
#                     (MainDictEmbedding[ind]['hypernyms']).append(int(np.mean(model2[i][0]*1000)))
#             MainDictEmbedding[ind]['hypernyms']=np.mean(MainDictEmbedding[ind]['hypernyms'])
            
                
            
            
            tempLemma=[]
            lemmatizer = WordNetLemmatizer()
            for i in tokens:
                tempLemma.append(lemmatizer.lemmatize(i))
            lemmaLi=[]
            lemmaLi.append(tempLemma)
            model1 = Word2Vec(lemmaLi, min_count=1,size=1)
            words1 = list(model1.wv.vocab)
            for i in words1:
                if i in tempLemma:
                    (MainDictEmbedding[ind]['lemma']).append(int(model1[i][0]*1000))
            MainDictEmbedding[ind]['lemma']=np.std(MainDictEmbedding[ind]['lemma'])
            
            

            for k1,v1 in MainDict1[ind]['POSTAG'].items():
                if v1 in posEmbDict.keys():
                    (MainDictEmbedding[ind]['POSTAG']).append(posEmbDict[v1])
                else:
                    (MainDictEmbedding[ind]['POSTAG']).append(0)
            MainDictEmbedding[ind]['POSTAG']=np.std(MainDictEmbedding[ind]['POSTAG'])

            for k1,v1 in MainDict1[ind]['NER'].items():
                if v1 in nerTagEmbDict.keys():
                    (MainDictEmbedding[ind]['NER']).append(nerTagEmbDict[v1])
                else:
                    (MainDictEmbedding[ind]['NER']).append(0)
            MainDictEmbedding[ind]['NER']=np.std(MainDictEmbedding[ind]['NER'])

            for k1,v1 in MainDict1[ind]['DEPPARSE'].items():
                if v1 in depParseEmbDict.keys():
                    (MainDictEmbedding[ind]['DEPPARSE']).append(depParseEmbDict[v1])
                else:
                    (MainDictEmbedding[ind]['DEPPARSE']).append(0)
            MainDictEmbedding[ind]['DEPPARSE']=np.std(MainDictEmbedding[ind]['DEPPARSE'])        

            if MainDict1[ind]['REL'] in reldict:
                MainDictEmbedding[ind]['REL']=reldict[MainDict1[ind]['REL']]

            MainDictEmbedding[ind]['Direction']=int(MainDict1[ind]['Direction'])
            MainDictEmbedding[ind]['e1Start']=int(MainDict1[ind]['e1Start'])
            MainDictEmbedding[ind]['e1End']=int(MainDict1[ind]['e1End'])
            MainDictEmbedding[ind]['e2Start']=int(MainDict1[ind]['e2Start'])
            MainDictEmbedding[ind]['e2End']=int(MainDict1[ind]['e2End'])
            ind+=1
            
    
    df= pd.DataFrame.from_dict(MainDictEmbedding)

    return(df,relList,dirList,wholeRelList)




In [6]:
import re 
import nltk
f2=open(r"test_data1.txt")
f1=open(r"train_data1.txt")
corpus_dict1=Corpus_dictFormation(f1)
corpus_dict2=Corpus_dictFormation(f2)

df1,relList1,dirList1,wholeRelList1=Corpus_Reading(corpus_dict1)
df2,relList2,dirList2,wholeRelList2=Corpus_Reading(corpus_dict2)


[2 0 3 6 1 7 5 8 9 4]


C:\Users\prayaas\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\prayaas\Anaconda3\lib\site-packages\ipykernel_launcher.py:205: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[2 0 3 6 1 7 5 8 9 4]


In [16]:
from sklearn import svm



Train_X = np.asarray(df1.loc[['POSTAG', 'NER', 'DEPPARSE','Direction','lemma']])
Train_Y = np.atleast_2d(np.asarray(df1.loc['REL']))
Train_X=Train_X.T
Train_Y=Train_Y.T
# Train_Y=Train_Y.astype('int')

Test_X = np.asarray(df2.loc[['POSTAG', 'NER', 'DEPPARSE','Direction','lemma']])
Test_Y = np.atleast_2d(np.asarray(df2.loc['REL']))
Test_X=Test_X.T
Test_Y=Test_Y.T

clf = svm.SVC()
clf.fit(Train_X,Train_Y)

y_predict=np.atleast_2d(clf.predict(Test_X))
# print(l)
# print(y_predict)

# print(clf.score(l,Test_Y.T))
from sklearn.metrics import accuracy_score
print(accuracy_score(Test_Y, y_predict.T))
print(Test_Y)

C:\Users\prayaas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\prayaas\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.29970544918998526
[[7.]
 [9.]
 [5.]
 ...
 [9.]
 [1.]
 [9.]]


In [17]:
relDict = REL_embedding(corpus_dict1)
count =0
for v1 in y_predict[0]:
    for k,v in relDict.items():
        if v1 == v:
#             print(k)
            count +=1
print(count)

[2 0 3 6 1 7 5 8 9 4]
2716


In [16]:
relList2Encode =[]
for l in relList2:
    if l in relDict.keys():
        relList2Encode.append(relDict[l])
    else:
        relList2Encode.append(10)
        
# print(relList2Encode)
print(len(relList2Encode))

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(relList2Encode, y_predict.T)

0.20655375552282768

In [20]:
from sklearn import svm
from sklearn import linear_model


Train_X1 = np.asarray(df1.loc[['POSTAG', 'NER', 'DEPPARSE','REL','lemma']])
Train_Y1 = np.atleast_2d(np.asarray(df1.loc['Direction']))
Train_X1=Train_X1.T
Train_Y1=Train_Y1.T


Test_X1 = np.asarray(df2.loc[['POSTAG', 'NER', 'DEPPARSE','REL','lemma']])
Test_Y1 = np.atleast_2d(np.asarray(df2.loc['Direction']))
Test_X1=Test_X1.T
Test_Y1=Test_Y1.T

clf1 = svm.SVC()
clf1.fit(Train_X1,Train_Y1)

y_predict1=np.atleast_2d(clf1.predict(Test_X1))
# print(l)
print(y_predict1)

C:\Users\prayaas\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\prayaas\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[[1. 0. 1. ... 0. 1. 0.]]


In [21]:
from sklearn.metrics import accuracy_score
print(accuracy_score(dirList2, y_predict1.T))
count=0
predictedRelList = []
for v1 in range(len(y_predict[0])):
    for k,v in relDict.items():
        if y_predict[0][v1] == v and y_predict1[0][v1] == 1:
            predictedRelList.append(str(k+"(e1,e2)"))
            
        elif y_predict[0][v1] == v and y_predict1[0][v1] == -1:
            predictedRelList.append(str(k+"(e2,e1)"))
            
        elif y_predict[0][v1] == v and y_predict1[0][v1] == 0:
            predictedRelList.append(k)
            

        
for i in range(len(predictedRelList)):
    if(predictedRelList[i] == wholeRelList2[i]):
        count +=1
print(count/len(predictedRelList))        
        

0.6649484536082474
0.164580265095729


In [12]:
print(df1.head)

<bound method NDFrame.head of               0        1        2      3        4        5        6     \
token      39.3529 -35.3125 -54.2667 -149.5 -29.8125  14.3846  7.96667   
POSTAG     9.82353  10.5625   9.8125  6.375  8.52381       14  8.48649   
NER        10.8824   10.875  10.8667  10.75   10.875  10.8462  10.9333   
DEPPARSE   22.7647   25.625     25.8  24.75       25  24.9231     21.1   
Direction       -1        0       -1      0        1        0       -1   
REL              1        8        5      8        6        8        0   
e1Start         12        1        1      2        1        4        7   
e1End           12        1        1      2        1        4        7   
e2Start         15        9        7      6        2       10       19   
e2End           15        9        7      6        2       10       19   
lemma           41    -43.5 -7.73333 -149.5 -29.8125  14.3846    -37.1   
hypernyms       []       []       []     []       []       []       []   

       

In [41]:
# x=[]
# for i in ['dog']:
#     y=find_hypernyms(i) 
#     x.append(y)
# print(x)

[['chromatic_color.n.01', 'tract.n.01', 'environmentalist.n.01', 'site.n.01', 'vegetable.n.01', 'ketamine.n.01', 'discolor.v.03']]


In [40]:
# hyperEmb1=x
# res=[]

# # print(hyperEmb1)
# for i in hyperEmb1:
#     some=[]
    
#     some.append(i)
#     print(some)
#     modelh = Word2Vec(some, min_count=1,size=1)
#     wordsh = list(modelh.wv.vocab)
#     for i in wordsh:
#         if i in hyperEmb1:
#             res.append(int(np.mean(modelh[i][0]*1000)))
# ans=np.mean(res)
# print(ans)

[['chromatic_color.n.01', 'tract.n.01', 'environmentalist.n.01', 'site.n.01', 'vegetable.n.01', 'ketamine.n.01', 'discolor.v.03']]
[['chromatic_color.n.01', 'tract.n.01', 'environmentalist.n.01', 'site.n.01', 'vegetable.n.01', 'ketamine.n.01', 'discolor.v.03']]
[['chromatic_color.n.01', 'tract.n.01', 'environmentalist.n.01', 'site.n.01', 'vegetable.n.01', 'ketamine.n.01', 'discolor.v.03']]
[['chromatic_color.n.01', 'tract.n.01', 'environmentalist.n.01', 'site.n.01', 'vegetable.n.01', 'ketamine.n.01', 'discolor.v.03']]
nan


In [18]:
count1=0
predictedRelationList = []
for v1 in range(len(y_predict[0])):
    for k,v in relDict.items():
        if y_predict[0][v1] == v:
            predictedRelationList.append((k))
            
        elif y_predict[0][v1] == v :
            predictedRelationList.append(k)
            
        elif y_predict[0][v1] == v:
            predictedRelationList.append(k)
            
onlyrel=[]
for i in wholeRelList2:
    if i!="O":
        onlyrel.append(i.split("(")[0])
    else:
        onlyrel.append("O")
        

for i in range(len(wholeRelList2)):
    if onlyrel[i]==predictedRelationList[i]:
        count1+=1
print(count1/len(predictedRelationList))
    

0.1800441826215022
